In [1]:
import daft
from daft import col
import json
import time
from openai import OpenAI
from dotenv import load_dotenv
from typing import List

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI()

In [2]:
df = daft.read_json("wikipedia_articles.jsonl")

╭───────────────────┬───────────────────┬───────────────────┬───────────────────┬───────────────────┬──────────────────╮
│ title             ┆ summary           ┆ text              ┆ url               ┆ categories        ┆ links            │
│ ---               ┆ ---               ┆ ---               ┆ ---               ┆ ---               ┆ ---              │
│ Utf8              ┆ Utf8              ┆ Utf8              ┆ Utf8              ┆ List[Utf8]        ┆ List[Utf8]       │
╞═══════════════════╪═══════════════════╪═══════════════════╪═══════════════════╪═══════════════════╪══════════════════╡
│ Chris Pratt       ┆ Christopher       ┆ Christopher       ┆ https://en.wikipe ┆ [Category:1979    ┆ [10 Years (2011  │
│                   ┆ Michael Pratt     ┆ Michael Pratt     ┆ dia.org/wiki…     ┆ births, Catego…   ┆ film), 1st Cr…   │
│                   ┆ (bo…              ┆ (bo…              ┆                   ┆                   ┆                  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Pope Anastasius I ┆ Pope Anastasius I ┆ Pope Anastasius I ┆ https://en.wikipe ┆ [Category:401     ┆ [19 martyrs of   │
│                   ┆ was the bis…      ┆ was the bis…      ┆ dia.org/wiki…     ┆ deaths, Categor…  ┆ Algeria, 2015 …  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Vancouver         ┆ Vancouver         ┆ Vancouver         ┆ https://en.wikipe ┆ [Category:2009    ┆ [1974 Vancouver  │
│ Whitecaps FC      ┆ Whitecaps         ┆ Whitecaps         ┆ dia.org/wiki…     ┆ establishments…   ┆ Whitecaps sea…   │
│                   ┆ Football …        ┆ Football …        ┆                   ┆                   ┆                  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Green, Green      ┆ "Green, Green     ┆ "Green, Green     ┆ https://en.wikipe ┆ [Category:1965    ┆ [(Everything I   │
│ Grass of Home     ┆ Grass of Home",…  ┆ Grass of Home",…  ┆ dia.org/wiki…     ┆ songs, Categor…   ┆ Do) I Do It fo…  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Siege of Wasit    ┆ The siege of      ┆ The siege of      ┆ https://en.wikipe ┆ [Category:740s    ┆ [Abbasid         │
│                   ┆ Wasit involved t… ┆ Wasit involved t… ┆ dia.org/wiki…     ┆ conflicts, Cat…   ┆ Caliphate,       │
│                   ┆                   ┆                   ┆                   ┆                   ┆ Abbasid R…       │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ …                 ┆ …                 ┆ …                 ┆ …                 ┆ …                 ┆ …                │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Imperial          ┆ The Imperial      ┆ The Imperial      ┆ https://en.wikipe ┆ [Category:All     ┆ [Chief Justice   │
│ Investiture       ┆ Investiture       ┆ Investiture       ┆ dia.org/wiki…     ┆ Wikipedia artic…  ┆ of Japan, Chiy…  │
│                   ┆ (親任式…          ┆ (親任式…          ┆                   ┆                   ┆                  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ European          ┆ During the Age of ┆ During the Age of ┆ https://en.wikipe ┆ [Category:Age of  ┆ [1491: New       │
│ colonization of   ┆ Discovery, …      ┆ Discovery, …      ┆ dia.org/wiki…     ┆ Discovery, C…     ┆ Revelations of   │
│ the …             ┆                   ┆                   ┆                   ┆                   ┆ the…             │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Saturn     

In [3]:
# add a new column to the dataframe that is randomly true or false

@daft.udf(batch_size=100, return_dtype=daft.DataType.bool())
def is_entity(title: daft.Series) -> List[bool]:
    """
    Determine if each title represents a person or organization entity.
    Uses OpenAI API with batched requests and retries for missing responses.
    
    Args:
        title: Series of article titles (guaranteed to be 100 or fewer)
        
    Returns:
        List of boolean values indicating if each title is an entity
    """
    # Load environment variables
    load_dotenv()
    
    # Initialize OpenAI client
    client = OpenAI()
    
    # Convert series to Python lists
    titles_list = title.to_pylist()
    
    # Track results
    results_dict = {}  # Map of title to is_entity result
    
    # Function to classify a batch of titles
    def classify_batch(batch_titles):
        try:
            # Create the input text with all titles
            titles_text = ", ".join(batch_titles)
            
            # Create response stream
            response_stream = client.responses.create(
                model="gpt-4o-mini",
                input=[
                    {
                        "role": "system",
                        "content": [
                            {
                                "type": "input_text",
                                "text": "You are given a list of topic titles. For each topic, determine whether it refers to a person or an organization. If it does, set \"is_entity\" to true; otherwise, set \"is_entity\" to false. Return your answer as a valid JSON array, where each element has the format:\n\n{\n\"title\": \"<TOPIC_TITLE>\",\n\"is_entity\": <true_or_false>\n}\n\nDo not return any additional keys, text, or commentary. Include ALL the titles I provide in your response."
                            }
                        ]
                    },
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "input_text",
                                "text": titles_text
                            }
                        ]
                    }
                ],
                text={
                    "format": {
                        "type": "text"
                    }
                },
                reasoning={},
                tools=[],
                temperature=0,
                max_output_tokens=10000,
                top_p=0,
                stream=True,
                store=True
            )
            
            # Collect the response
            response_text = ""
            for chunk in response_stream:
                if hasattr(chunk, 'text') and chunk.text:
                    response_text += chunk.text
            
            # Extract JSON from the response (remove markdown code blocks if present)
            if "```json" in response_text:
                json_text = response_text.split("```json")[1].split("```")[0].strip()
            elif "```" in response_text:
                json_text = response_text.split("```")[1].split("```")[0].strip()
            else:
                json_text = response_text.strip()
            
            # Parse JSON
            batch_results = json.loads(json_text)
            
            # Convert to dictionary for easier lookup
            return {item["title"]: item["is_entity"] for item in batch_results}
            
        except Exception as e:
            print(f"Error in batch classification: {e}")
            # Return empty dict on error to trigger retry
            return {}
    
    # Since we know it's 100 or fewer titles, we can process all at once
    remaining_titles = titles_list.copy()
    
    # Maximum retry attempts
    max_retries = 3
    
    # Keep querying until we have results for all titles or exceed max retries
    retry_count = 0
    while remaining_titles and retry_count < max_retries:
        # Classify all remaining titles
        batch_results = classify_batch(remaining_titles)
        
        # Update results dictionary with successful classifications
        results_dict.update(batch_results)
        
        # Determine which titles still need classification
        processed_titles = set(batch_results.keys())
        remaining_titles = [t for t in remaining_titles if t not in processed_titles]
        
        # If we still have unprocessed titles, increment retry counter
        if remaining_titles:
            retry_count += 1
            # Wait briefly before retrying to avoid rate limits
            time.sleep(1)
    
    # For any remaining unclassified titles after max retries, default to False
    for title_text in remaining_titles:
        results_dict[title_text] = False
    
    # Return results in the original order
    return [results_dict.get(t, False) for t in titles_list]

is_entity_4_concurrency = is_entity.with_concurrency(4)
df = df.with_column("is_entity", is_entity_4_concurrency(df["title"]))

In [4]:
df.show(100)

╭──────────────────┬──────────────────┬──────────────────┬────────────┬──────────────────┬─────────────────┬───────────╮
│ title            ┆ summary          ┆ text             ┆      …     ┆ categories       ┆ links           ┆ is_entity │
│ ---              ┆ ---              ┆ ---              ┆            ┆ ---              ┆ ---             ┆ ---       │
│ Utf8             ┆ Utf8             ┆ Utf8             ┆ (1 hidden) ┆ List[Utf8]       ┆ List[Utf8]      ┆ Boolean   │
╞══════════════════╪══════════════════╪══════════════════╪════════════╪══════════════════╪═════════════════╪═══════════╡
│ Chris Pratt      ┆ Christopher      ┆ Christopher      ┆ …          ┆ [Category:1979   ┆ [10 Years (2011 ┆ true      │
│                  ┆ Michael Pratt    ┆ Michael Pratt    ┆            ┆ births, Catego…  ┆ film), 1st Cr…  ┆           │
│                  ┆ (bo…             ┆ (bo…             ┆            ┆                  ┆                 ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ Pope Anastasius  ┆ Pope Anastasius  ┆ Pope Anastasius  ┆ …          ┆ [Category:401    ┆ [19 martyrs of  ┆ true      │
│ I                ┆ I was the bis…   ┆ I was the bis…   ┆            ┆ deaths, Categor… ┆ Algeria, 2015 … ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ Vancouver        ┆ Vancouver        ┆ Vancouver        ┆ …          ┆ [Category:2009   ┆ [1974 Vancouver ┆ true      │
│ Whitecaps FC     ┆ Whitecaps        ┆ Whitecaps        ┆            ┆ establishments…  ┆ Whitecaps sea…  ┆           │
│                  ┆ Football …       ┆ Football …       ┆            ┆                  ┆                 ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ Green, Green     ┆ "Green, Green    ┆ "Green, Green    ┆ …          ┆ [Category:1965   ┆ [(Everything I  ┆ false     │
│ Grass of Home    ┆ Grass of Home",… ┆ Grass of Home",… ┆            ┆ songs, Categor…  ┆ Do) I Do It fo… ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ Siege of Wasit   ┆ The siege of     ┆ The siege of     ┆ …          ┆ [Category:740s   ┆ [Abbasid        ┆ false     │
│                  ┆ Wasit involved   ┆ Wasit involved   ┆            ┆ conflicts, Cat…  ┆ Caliphate,      ┆           │
│                  ┆ t…               ┆ t…               ┆            ┆                  ┆ Abbasid R…      ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ …                ┆ …                ┆ …                ┆ …          ┆ …                ┆ …               ┆ …         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ Template         ┆                  ┆                  ┆ …          ┆ [Category:NA-imp ┆ [Europe, Wikipe ┆ false     │
│ talk:CIS leaders ┆                  ┆                  ┆            ┆ ortance Europ…   ┆ dia:Content as… ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ The Shard        ┆ The Shard, also  ┆ The Shard, also  ┆ …          ┆ [Category:All    ┆ [100            ┆ false     │
│                  ┆ referred to a…   ┆ referred to a…   ┆            ┆ articles with u… ┆ Bishopsgate,    ┆           │
│                  ┆                  ┆                  ┆            ┆                  ┆ 100 Greenga…    ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ Love Yourself in ┆ Love Yourself in ┆ Love Yourself in ┆ …          ┆ [Category:2010s  ┆ [2 Cool 4       ┆ false     │
│ Seoul

In [5]:
# print title and is_entity to json

df_title_is_entity = df.select("title", "is_entity").collect()

with open("entity_classifier.json", "w") as f:
    json.dump(df_title_is_entity.to_pylist(), f)

🗡️ 🐟 ActorPoolProject: 20:55 18,752 rows received, 18,300 rows emittedError in batch classification: peer closed connection without sending complete message body (incomplete chunked read)


In [6]:
# # filter the dataframe to only include rows where is_entity is true

# TODO
df = df.where(df["is_entity"])
df = df.collect()

╭──────────────────┬──────────────────┬──────────────────┬────────────┬──────────────────┬─────────────────┬───────────╮
│ title            ┆ summary          ┆ text             ┆      …     ┆ categories       ┆ links           ┆ is_entity │
│ ---              ┆ ---              ┆ ---              ┆            ┆ ---              ┆ ---             ┆ ---       │
│ Utf8             ┆ Utf8             ┆ Utf8             ┆ (1 hidden) ┆ List[Utf8]       ┆ List[Utf8]      ┆ Boolean   │
╞══════════════════╪══════════════════╪══════════════════╪════════════╪══════════════════╪═════════════════╪═══════════╡
│ Chris Pratt      ┆ Christopher      ┆ Christopher      ┆ …          ┆ [Category:1979   ┆ [10 Years (2011 ┆ true      │
│                  ┆ Michael Pratt    ┆ Michael Pratt    ┆            ┆ births, Catego…  ┆ film), 1st Cr…  ┆           │
│                  ┆ (bo…             ┆ (bo…             ┆            ┆                  ┆                 ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ Pope Anastasius  ┆ Pope Anastasius  ┆ Pope Anastasius  ┆ …          ┆ [Category:401    ┆ [19 martyrs of  ┆ true      │
│ I                ┆ I was the bis…   ┆ I was the bis…   ┆            ┆ deaths, Categor… ┆ Algeria, 2015 … ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ Vancouver        ┆ Vancouver        ┆ Vancouver        ┆ …          ┆ [Category:2009   ┆ [1974 Vancouver ┆ true      │
│ Whitecaps FC     ┆ Whitecaps        ┆ Whitecaps        ┆            ┆ establishments…  ┆ Whitecaps sea…  ┆           │
│                  ┆ Football …       ┆ Football …       ┆            ┆                  ┆                 ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ Empire of Japan  ┆ The Empire of    ┆ The Empire of    ┆ …          ┆ [Category:1868   ┆ [1899–1923      ┆ true      │
│                  ┆ Japan, also kno… ┆ Japan, also kno… ┆            ┆ establishments…  ┆ cholera         ┆           │
│                  ┆                  ┆                  ┆            ┆                  ┆ pandemic, …     ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ ISABELLE         ┆ ISABELLE (also   ┆ ISABELLE (also   ┆ …          ┆ [Category:Brookh ┆ [ADA collider,  ┆ true      │
│                  ┆ known later as…  ┆ known later as…  ┆            ┆ aven National…   ┆ Bibcode (ident… ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ …                ┆ …                ┆ …                ┆ …          ┆ …                ┆ …               ┆ …         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ Chuvash          ┆ The Chuvash      ┆ The Chuvash      ┆ …          ┆ [Category:All    ┆ [Adyghe         ┆ true      │
│ Wikipedia        ┆ Wikipedia        ┆ Wikipedia        ┆            ┆ Wikipedia artic… ┆ Wikipedia,      ┆           │
│                  ┆ (Chuvas…         ┆ (Chuvas…         ┆            ┆                  ┆ Afrikaans …     ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ Joseph Cross     ┆ Joseph Michael   ┆ Joseph Michael   ┆ …          ┆ [Category:1986   ┆ [19th Youth in  ┆ true      │
│ (actor)          ┆ Cross (born Ma…  ┆ Cross (born Ma…  ┆            ┆ births, Catego…  ┆ Film Awards, 2… ┆           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ National         ┆ National         ┆ National         ┆ …          ┆ [Category:1888   ┆ [135 film, 1888 ┆ true      │
│ Geogr

In [7]:
# drop categories and is_entity column
df = df.exclude("categories", "is_entity")
df = df.collect()

╭───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬──────────────────────╮
│ title                 ┆ summary               ┆ text                  ┆ url                   ┆ links                │
│ ---                   ┆ ---                   ┆ ---                   ┆ ---                   ┆ ---                  │
│ Utf8                  ┆ Utf8                  ┆ Utf8                  ┆ Utf8                  ┆ List[Utf8]           │
╞═══════════════════════╪═══════════════════════╪═══════════════════════╪═══════════════════════╪══════════════════════╡
│ Chris Pratt           ┆ Christopher Michael   ┆ Christopher Michael   ┆ https://en.wikipedia. ┆ [10 Years (2011      │
│                       ┆ Pratt (bo…            ┆ Pratt (bo…            ┆ org/wiki…             ┆ film), 1st Cr…       │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Pope Anastasius I     ┆ Pope Anastasius I was ┆ Pope Anastasius I was ┆ https://en.wikipedia. ┆ [19 martyrs of       │
│                       ┆ the bis…              ┆ the bis…              ┆ org/wiki…             ┆ Algeria, 2015 …      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Vancouver Whitecaps   ┆ Vancouver Whitecaps   ┆ Vancouver Whitecaps   ┆ https://en.wikipedia. ┆ [1974 Vancouver      │
│ FC                    ┆ Football …            ┆ Football …            ┆ org/wiki…             ┆ Whitecaps sea…       │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Empire of Japan       ┆ The Empire of Japan,  ┆ The Empire of Japan,  ┆ https://en.wikipedia. ┆ [1899–1923 cholera   │
│                       ┆ also kno…             ┆ also kno…             ┆ org/wiki…             ┆ pandemic, …          │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Quintus Dellius       ┆ Quintus Dellius       ┆ Quintus Dellius       ┆ https://en.wikipedia. ┆ [Alexander Helios,   │
│                       ┆ (Ancient Gree…        ┆ (Ancient Gree…        ┆ org/wiki…             ┆ Alexandria…          │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ …                     ┆ …                     ┆ …                     ┆ …                     ┆ …                    │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Royal Spanish Academy ┆ The Royal Spanish     ┆ The Royal Spanish     ┆ https://en.wikipedia. ┆ [2010 Chile          │
│                       ┆ Academy (Sp…          ┆ Academy (Sp…          ┆ org/wiki…             ┆ earthquake, Acade…   │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Haji Özbek Mosque     ┆ Haji Özbek Mosque     ┆ Haji Özbek Mosque     ┆ https://en.wikipedia. ┆ [Ahmet Hamdi Akseki  │
│                       ┆ (Turkish: H…          ┆ (Turkish: H…          ┆ org/wiki…             ┆ Mosque, A…           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Stereophonics         ┆ Stereophonics are a   ┆ Stereophonics are a   ┆ https://en.wikipedia. ┆ [A1 (group), A Day   │
│                       ┆ Welsh roc…            ┆ Welsh roc…            ┆ org/wiki…             ┆ at the Rac…          │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Federal Reserve Bank  ┆ The Federal Reserve   ┆ The Federal Reserve   ┆ https://en.wikipedia. ┆ [1933 Banking Act,   │
│ of New Y…             ┆ Bank of N…            ┆ Bank of N…            ┆ org/wiki…             ┆ 2009 Super…          │
├╌╌╌╌╌╌

















































































































































































































Error in batch classification: peer closed connection without sending complete message body (incomplete chunked read)































































































































































In [8]:
df_with_text = df.select("title", "summary", "text", "url")
df = df.exclude("summary", "text", "url")

df = df.explode("links")
df = df.collect()

In [9]:
print(df.count_rows())
df = df.join(df, how="semi", left_on="links", right_on="title").collect()
print(df.count_rows())
df = df.groupby("title").agg(col("links").agg_list().alias("links"))
df = df.join(df_with_text, how="left", left_on="title", right_on="title")
df = df.collect()

3692602
305921


In [10]:
# CELL 1: Generate embeddings for all article titles
@daft.udf(batch_size=100, return_dtype=daft.DataType.python())
def generate_title_embeddings(title: daft.Series) -> List[List[float]]:
    """
    Generate embeddings for article titles using OpenAI's embedding model.
    
    Args:
        title: Series of article titles
        
    Returns:
        List of embedding vectors
    """
    from openai import OpenAI
    from dotenv import load_dotenv
    import time
    
    # Load environment variables
    load_dotenv()
    client = OpenAI()
    
    titles = title.to_pylist()
    embeddings = []
    
    # Process in batches of 20 to avoid rate limits
    batch_size = 20
    for i in range(0, len(titles), batch_size):
        batch = titles[i:i+batch_size]
        
        try:
            response = client.embeddings.create(
                input=batch,
                model="text-embedding-3-small"
            )
            
            # Extract embeddings from response
            for embedding_data in response.data:
                embeddings.append(embedding_data.embedding)
                
        except Exception as e:
            print(f"Error generating embeddings for batch {i}:{i+batch_size}: {e}")
            # Add empty embeddings for failed batch
            for _ in range(len(batch)):
                embeddings.append([])
        
        # Small pause to avoid rate limits
        if i + batch_size < len(titles):
            time.sleep(0.1)
    
    return embeddings

# Add title embeddings to the dataframe
print("Generating embeddings for all article titles...")
df_with_text = df_with_text.with_column("title_embedding", generate_title_embeddings(df_with_text["title"]))
print("Title embeddings generated.")
df_with_text = df_with_text.collect()

Generating embeddings for all article titles...
Title embeddings generated.


In [11]:
print("Creating article-link pairs...")
df_exploded = df.explode("links")
print(f"Generated {df_exploded.count_rows()} article-link pairs")
df_exploded = df_exploded.collect()

Creating article-link pairs...
Generated 317610 article-link pairs


In [12]:
print("Adding source title embeddings...")
df_exploded = df_exploded.join(
    df_with_text.select("title", "title_embedding"),
    left_on="title",
    right_on="title",
    how="left"
)
df_exploded = df_exploded.collect()

Adding source title embeddings...


In [13]:
# print("Adding target link embeddings...")
# df_exploded = df_exploded.join(
#     df_with_text.select("title", "title_embedding"),
#     left_on="links",
#     right_on="title",
#     how="left",
#     right_select=["title_embedding"]
# ).rename("title_embedding_right", "link_embedding")
# df_exploded.show(10)

df_exploded = df_exploded.join(
    df_with_text,
    left_on="links",
    right_on="title",
    how="left",
    suffix="_right"
).with_column_renamed("title_embedding_right", "link_embedding")
df_exploded = df_exploded.collect()

In [14]:
@daft.udf(batch_size=1000, return_dtype=daft.DataType.float64())
def calculate_cosine_similarity(source_embedding: daft.Series, target_embedding: daft.Series) -> List[float]:
    """
    Calculate cosine similarity between source and target embeddings.
    
    Args:
        source_embedding: Series of source embeddings
        target_embedding: Series of target embeddings
        
    Returns:
        List of similarity scores
    """
    from scipy.spatial.distance import cosine
    
    source_embeddings = source_embedding.to_pylist()
    target_embeddings = target_embedding.to_pylist()
    similarities = []
    
    for src_emb, tgt_emb in zip(source_embeddings, target_embeddings):
        # Handle empty embeddings
        if not src_emb or not tgt_emb:
            similarities.append(0.0)
            continue
            
        # Calculate cosine similarity
        # Convert distance to similarity (1 - distance)
        similarity = 1.0 - cosine(src_emb, tgt_emb)
        similarities.append(similarity)
    
    return similarities

print("Calculating cosine similarities...")
df_exploded = df_exploded.with_column(
    "similarity", 
    calculate_cosine_similarity(df_exploded["title_embedding"], df_exploded["link_embedding"])
)
df_exploded = df_exploded.collect()

Calculating cosine similarities...


In [15]:
print("Filtering valid pairs...")
df_exploded = df_exploded.where(df_exploded["similarity"] > 0)
print(f"Remaining pairs after filtering: {df_exploded.count_rows()}")
df_exploded = df_exploded.collect()

Filtering valid pairs...
Remaining pairs after filtering: 352931


In [16]:
# print("Selecting top 10 neighbors for each article...")
# neighbors_df = df_exploded.groupby("title").agg(
#     df_exploded.sort("similarity", desc=True).limit(10)
# )
# print(f"Generated neighbors for {neighbors_df.count_rows()} articles")
# neighbors_df.show(5)

sorted_df = df_exploded.sort("similarity", desc=True)
# Group by 'title' and use a custom aggregation to get the top 10 neighbors
neighbors_df = sorted_df.groupby("title").agg(
    col("links").agg_list().alias("top_links")
).with_column("top_links", col("top_links").list.slice(0, 10))
# Materialize the DataFrame to count rows
neighbors_df.collect()
print(f"Generated neighbors for (neighbors_df.count_rows()) articles")
neighbors_df = neighbors_df.collect()

Generated neighbors for (neighbors_df.count_rows()) articles


In [17]:
print("Preparing node data for parquet file...")

# Create a nodes dataframe with all the relevant attributes
nodes_df = df_with_text.select(
    col("title").alias("id"),         # Node ID
    "summary",       # Short description
    "text",          # Full content
)

print(f"Writing {nodes_df.count_rows()} nodes to parquet...")
nodes_df.write_parquet("wikipedia_nodes.parquet")
print("Node data saved to wikipedia_nodes.parquet")

Preparing node data for parquet file...
Writing 7863 nodes to parquet...
Node data saved to wikipedia_nodes.parquet                  d


In [18]:
print("Preparing edge data for parquet file...")

# We'll use the exploded dataframe with similarity scores as our edge list
edge_df = df_exploded.select(
    "title",                 # Source node
    "links",                 # Target node
    "similarity"             # Edge weight
)

# Rearrange columns and rename for clarity
edge_df = edge_df.select(
    col("title").alias("source"),
    col("links").alias("target"),
    "similarity"
)

print(f"Writing {edge_df.count_rows()} edges to parquet...")
edge_df.write_parquet("wikipedia_edges.parquet")
print("Edge data saved to wikipedia_edges.parquet")

Preparing edge data for parquet file...
Writing 352931 edges to parquet...
Edge data saved to wikipedia_edges.parquet
